## 1. Processing Text, Vectorize Author List

# Author-Topic Modeling
### Source: https://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

In [17]:
import nltk
import json
content = []

content = []
with open('total.json') as json_data:
    content = json.load(json_data)

#for line in open('JACS.json', 'r'):
#    content.append(json.loads(line))
    
#print(len(content))


5676


In [18]:
# Get all author names and their corresponding document IDs.
author2doc = dict()

i = 0
for entry in content:
    sender = entry['Sender'].replace('\n',' ')
    if not author2doc.get(sender):
        # This is a new author.
        #author2doc[sender] = []
        author2doc[sender] = [i]
    # Add document IDs to author.
    else:
        author2doc[sender].append(i)
    i = i + 1
    
i = 0    
for entry in content:
    receiver = entry['Receiver'].replace('\n',' ')
    if not author2doc.get(receiver):
        # This is a new author.
        author2doc[receiver] = []
        author2doc[receiver] = [i]
    # Add document IDs to author.
    else:
        author2doc[receiver].append(i)
    i = i + 1
    
    
#print(author2doc)


In [19]:
import spacy
nlp = spacy.load('en')

In [21]:
import re
abstract = []
for entry in content:
    title = entry['Title'].replace('\n',' ')
    title = title.replace('/u',' ')
    #sender = entry['Sender'].replace('\n',' ')
    #receiver = entry['Receiver'].replace('\n',' ')
    abst = entry['Content'].replace('\n',' ')
    abst = abst.replace('/u',' ')
    entry_str = title+' '+abst
    entry_str = re.sub(r'\b\w{1,3}\b', '',entry_str)
    abstract.append(entry_str)
    
#print(abstract[10])

In [22]:
from nltk.corpus import stopwords
d = {}
stopword = stopwords.words('english')

In [23]:
%%time
processed_docs = []    
for doc in nlp.pipe(abstract, n_threads=4, batch_size=100):
    # Process document using Spacy NLP pipeline.
    
    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stopword]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

CPU times: user 5min 48s, sys: 1min, total: 6min 48s
Wall time: 4min 49s


In [24]:
abstract_all = processed_docs
del processed_docs

In [25]:
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(abstract_all, min_count=20)
for idx in range(len(abstract_all)):
    for token in bigram[abstract_all[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            abstract_all[idx].append(token)

/Users/Maggie/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [26]:
from gensim.corpora import Dictionary
dictionary = Dictionary(abstract_all)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.2
min_wordcount = 80
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.

In [27]:
corpus = [dictionary.doc2bow(doc) for doc in abstract_all]

In [28]:
#print('Number of authors: %d' % len(author2doc))
#print('Number of unique tokens: %d' % len(dictionary))
#print('Number of documents: %d' % len(corpus))

Number of authors: 10345
Number of unique tokens: 1156
Number of documents: 5676


In [248]:
#print(len(corpus))
#print(len(dictionary.id2token))

## 2. Select Best Performing Model with highest coherence

## LDA-tfidf

In [30]:
# Make Corpus a tfidf corpus

from operator import itemgetter
import gensim
from gensim import corpora,models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

l = list()
for t in corpus_tfidf:
    l.append(t)
    

index = 0
l_rounded = []
for i in l:
    index +=1
    common_denom = min(i,key=itemgetter(1))[1] if i else None
    if common_denom is not None:
        new_list = []
        for f in i:
            n = f[1]/common_denom
            new_list.append((f[0],int(n)))
        l_rounded.append(new_list)
    else:
        #print(index)
        new_list = []
        for f in i:
            new_list.append(f[0],f[1])
        l_rounded.append(new_list)
    
#print(l_rounded[46])
#print(corpus[46])
#print(len(corpus))

[(12, 4), (36, 1), (60, 2), (61, 1), (67, 1), (79, 1), (80, 4), (87, 1), (111, 1), (112, 1), (116, 1), (125, 2), (128, 1), (137, 2), (147, 1), (155, 1), (159, 2), (173, 1), (181, 4), (188, 2), (221, 1), (226, 1), (255, 1), (278, 8), (358, 1), (389, 4), (390, 4), (488, 1), (490, 1), (492, 14), (495, 14), (527, 1), (577, 2), (608, 1), (653, 2), (688, 11), (704, 1), (706, 1), (721, 1), (770, 2), (771, 2), (772, 1), (773, 2), (774, 1), (775, 2), (776, 2), (777, 2), (778, 2), (779, 2), (780, 2), (781, 2), (782, 2)]
[(12, 2), (36, 1), (60, 1), (61, 1), (67, 1), (79, 1), (80, 3), (87, 1), (111, 1), (112, 1), (116, 1), (125, 1), (128, 1), (137, 1), (147, 1), (155, 1), (159, 2), (173, 1), (181, 3), (188, 1), (221, 1), (226, 1), (255, 1), (278, 4), (358, 1), (389, 2), (390, 2), (488, 1), (490, 1), (492, 6), (495, 6), (527, 1), (577, 1), (608, 1), (653, 1), (688, 5), (704, 1), (706, 1), (721, 1), (770, 1), (771, 1), (772, 1), (773, 1), (774, 1), (775, 1), (776, 1), (777, 1), (778, 1), (779, 1), (

In [131]:
%%time
from gensim.models import AuthorTopicModel
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=l_rounded, num_topics=15, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(l_rounded)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 10min 47s, sys: 42.5 s, total: 11min 29s
Wall time: 11min 48s


In [133]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: -3.919e+01


In [33]:
model.save('/tmp/model.atmodel-tfidf')

In [34]:
model_tfidf = AuthorTopicModel.load('/tmp/model.atmodel-tfidf')

In [35]:
#model_tfidf.show_topics(num_topics=10)
topics = []
i = 1
for topic in model_tfidf.show_topics(num_topics=15):
    words = []
    for word, prob in model_tfidf.show_topic(topic[0]):
        words.append(word)
    #print('Topic '+str(i)+': ')
    #print(words[2]+' '+words[1]+' '+words[0])
    #print(*words)
    #print()
    i += 1
    topics.append('Topic '+str(i)+': '+words[2]+' '+words[1]+' '+words[0])

Topic 1: 
structure metal state
state metal structure organic material molecule assembly crystal chiral transition

Topic 2: 
protein network movement
movement network protein motor peptide signal membrane light simulation dynamic

Topic 3: 
choice risk decision
decision risk choice association community behavior research model article social

Topic 4: 
disorder therapy child
child therapy disorder infection resistance adult drug antibiotic mental old

Topic 5: 
acid catalyst reaction
reaction catalyst acid bond compound product carbon radical intermediate complex

Topic 6: 
specie plant gene
gene plant specie genetic genome sequence leaf variant trait photosynthesis

Topic 7: 
positive survival score
score survival positive test prognostic value negative predictive symptom factor

Topic 8: 
visual task experiment
experiment task visual image memory stimulus algorithm spatial learning processing

Topic 9: 
clinical cost trial
trial cost clinical treatment effectiveness outcome experien

## LDA

In [146]:
%%time
from gensim.models import AuthorTopicModel
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=corpus, num_topics=15, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    #using c_v score for coherence
    top_topics = model.top_topics(texts = abstract_all,dictionary = dictionary, coherence = 'c_v')
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 9min 44s, sys: 34.3 s, total: 10min 19s
Wall time: 11min 2s


In [147]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: 8.494e+00


In [148]:
model.save('/tmp/model.atmodel')

In [149]:
atmodel = AuthorTopicModel.load('/tmp/model.atmodel')

In [150]:
topics = []
i = 1
for topic in atmodel.show_topics(num_topics=15):
    words = []
    for word, prob in model.show_topic(topic[0]):
        words.append(word)
    print('Topic '+str(i)+': ')
    print(words[2]+' '+words[1]+' '+words[0])
    print(*words)
    print()
    i += 1
    topics.append(words[2]+' '+words[1]+' '+words[0])

Topic 1: 
complex protein reaction
reaction protein complex acid product synthesis catalyst bond ligand intermediate

Topic 2: 
structure specie surface
surface specie structure single interaction state molecular sequence energy site

Topic 3: 
drug tumor target
target tumor drug surgery image light area treatment imaging human

Topic 4: 
student medical research
research medical student development positive develop clinical process review focus

Topic 5: 
associate factor infection
infection factor associate rate prevalence hospital injury identify risk year

Topic 6: 
propose system model
model system propose paper application approach algorithm performance design information

Topic 7: 
visual task experiment
experiment task visual participant stimulus information memory processing spatial face

Topic 8: 
meta treatment trial
trial treatment meta meta_analysis risk therapy clinical outcome review cost

Topic 9: 
water material temperature
temperature material water property state org

In [151]:
from pprint import pprint

def show_author(name):
    print('\n%s' % name)
    print('Docs:', atmodel.author2doc[name])
    print('Topics:')
    pprint([(topics[topic[0]],topic[1]) for topic in atmodel[name]])

In [152]:
show_author('Jonathan L. Sessler')


Jonathan L. Sessler
Docs: [4005, 4139, 4308, 4706, 4809, 4811]
Topics:
[('complex protein reaction', 0.24220796911493042),
 ('structure specie surface', 0.24892795487591593),
 ('water material temperature', 0.5048779272833233)]


## LDA Mallet

In [153]:
mallet_path = '/Users/Maggie/Downloads/mallet-2.0.8/bin/mallet' # update this path
#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=dictionary.id2token)

In [154]:
from gensim.test.utils import datapath, get_tmpfile, common_texts
from gensim.corpora import MalletCorpus
from gensim.corpora import Dictionary

# Write corpus in Mallet format to disk
output_fname = get_tmpfile("corpus.mallet")
MalletCorpus.serialize(output_fname, corpus, dictionary)


mallet_corpus = MalletCorpus(output_fname)

In [155]:
print(mallet_corpus[10])
print(corpus[10])

[(268, 1), (737, 1), (63, 1), (169, 5), (490, 2), (509, 3), (716, 1), (473, 1), (545, 2), (193, 1), (210, 1), (329, 1), (334, 1), (380, 1), (549, 1), (738, 1), (815, 2), (987, 1), (1122, 2), (61, 1), (480, 2), (795, 1), (1019, 1), (856, 1), (1152, 1), (9, 1), (159, 1), (252, 3), (388, 2), (550, 1), (573, 1), (581, 4), (680, 1), (784, 1), (927, 1), (958, 1), (1103, 1)]
[(15, 1), (39, 1), (66, 1), (70, 5), (79, 2), (80, 3), (87, 1), (112, 1), (117, 2), (145, 1), (148, 1), (156, 1), (157, 1), (159, 1), (166, 1), (173, 1), (181, 2), (185, 1), (188, 2), (189, 1), (199, 2), (207, 1), (214, 1), (240, 1), (287, 1), (327, 1), (328, 1), (329, 3), (330, 2), (331, 1), (332, 1), (333, 4), (334, 1), (335, 1), (336, 1), (337, 1), (338, 1)]


In [156]:
mal = list()
for t in mallet_corpus:
    mal.append(t)

#print(mal[10])

[(268, 1), (737, 1), (63, 1), (169, 5), (490, 2), (509, 3), (716, 1), (473, 1), (545, 2), (193, 1), (210, 1), (329, 1), (334, 1), (380, 1), (549, 1), (738, 1), (815, 2), (987, 1), (1122, 2), (61, 1), (480, 2), (795, 1), (1019, 1), (856, 1), (1152, 1), (9, 1), (159, 1), (252, 3), (388, 2), (550, 1), (573, 1), (581, 4), (680, 1), (784, 1), (927, 1), (958, 1), (1103, 1)]


In [157]:
%%time
from gensim.models import AuthorTopicModel
model_list = []
for i in range(5):
    model = AuthorTopicModel(corpus=mal, num_topics=15, id2word=dictionary.id2token, \
                    author2doc=author2doc, chunksize=2000, passes=100, gamma_threshold=1e-10, \
                    eval_every=0, iterations=1, random_state=i)
    top_topics = model.top_topics(texts = abstract_all,dictionary = dictionary, coherence = 'c_v')
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

CPU times: user 10min 41s, sys: 35.5 s, total: 11min 16s
Wall time: 12min 14s


In [158]:
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

Topic coherence: 3.711e+00


In [159]:
model.save('/tmp/model.malletmodel')

In [160]:
malletmodel = AuthorTopicModel.load('/tmp/model.malletmodel')

In [161]:
topics = []
i = 1
for topic in malletmodel.show_topics(num_topics=15):
    words = []
    for word, prob in malletmodel.show_topic(topic[0]):
        words.append(word)
    print('Topic '+str(i)+': ')
    print(words[2]+' '+words[1]+' '+words[0])
    print(*words)
    print()
    i += 1
    topics.append(words[2]+' '+words[1]+' '+words[0])

Topic 1: 
valuable frequency region
region frequency valuable transcription -PRON- aim coupling current odd_ratio focus

Topic 2: 
redox express spectroscopy
spectroscopy express redox previous structural prior imaging recognition short uptake

Topic 3: 
woman pair validation
validation pair woman ring disease status enhance severity infection detection

Topic 4: 
implementation transcription deliver
deliver transcription implementation lower demand extract feasibility mouse stabilize resource

Topic 5: 
decision significantly_increase building
building significantly_increase decision traditional probe replicate developmental avoid help vary

Topic 6: 
strongly standard signaling
signaling standard strongly particularly carcinoma mechanistic probability goal vivo association

Topic 7: 
interface management solid
solid management interface world particularly nanoparticle %   liver focus interest

Topic 8: 
collection promising complete
complete promising collection particularly compound

## 3. Plotting the Authors

We take all the author-topic distributions (stored in model.state.gamma) and embed them in a 2D space. To do this, we reduce the dimensionality of this data using t-SNE.

t-SNE, or t-distributed stochastic neighbor embedding, is a dimensionality reduction algorithm for high-dimensional data visualization. This is partly to mitigate the fact that human cannot (at least not now) perceive vector space that is beyond 3-D.

t-SNE is a method that attempts to reduce the dimensionality of a dataset, while maintaining the distances between the points. That means that if two authors are close together in the plot below, then their topic distributions are similar.

In the cell below, we transform the author-topic representation into the t-SNE space. You can increase the smallest_author value if you do not want to view all the authors with few documents.



## Single-color Plot

In [66]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 2  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_


CPU times: user 16.7 s, sys: 1.38 s, total: 18.1 s
Wall time: 18.4 s


In [67]:
# Tell Bokeh to display plots inside the notebook.
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [162]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]



author_names = [model.id2author[a] for a in authors]

# Radius of each point corresponds to the number of documents attributed to that author.
scale = 0.4
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )


# Add author names and sizes to mouse-over info.
hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

mapper = linear_cmap(field_name='y', palette=Spectral6 ,low=-100 ,high=100)
p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_color = '#98df8a', fill_alpha=0.6, line_color=None)




show(p)

## Attempting: Color-coded Clusters

In [196]:
print(type(model.state.gamma))
print(model.state.gamma.argmax(axis=1))

<class 'numpy.ndarray'>
[ 1  9  7 ...  8 14  2]


In [164]:
colormap = [
    "#1f77b4", "#aec7e8", "#ff7f0e", "#ffbb78", "#2ca02c",
    "#98df8a", "#d62728", "#ff9896", "#9467bd", "#c5b0d5",
    "#8c564b", "#c49c94", "#e377c2", "#f7b6d2", "#7f7f7f",
    "#c7c7c7", "#bcbd22", "#dbdb8d", "#17becf", "#9edae5"]

In [165]:
from bokeh.palettes import all_palettes
from sklearn.cluster import KMeans
import numpy as np
import bokeh.plotting
from bokeh.plotting import figure
from sklearn import datasets


x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]

kmean = KMeans(n_clusters=15)

kmean.fit(tsne.embedding_)

colorchoice = [colormap[i] for i in kmean.labels_]
plot.circle(tsne.embedding_[:, 0], tsne.embedding_[:, 1], fill_color=colorchoice, line_color=None)
#p.scatter('x', 'y', radius='radii', source=source, fill_color = '#98df8a', fill_alpha=0.6, line_color=None)
show(plot)

NameError: name 'plot' is not defined

## 4. Similarity Queries

In [46]:
from gensim.similarities import MatrixSimilarity

# Generate a similarity object for the transformed corpus.
index = MatrixSimilarity(model[list(model.id2author.values())])

# Get similarities to some author.
author_name = 'Yadong Li'
sims = index[model[author_name]]

/Users/Maggie/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [47]:
# Make a function that returns similarities based on the Hellinger distance.

from gensim import matutils
import pandas as pd

# Make a list of all the author-topic distributions.
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    '''Get similarity between two vectors'''
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    '''Get similarity of vector to all authors.'''
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    '''
    Get table with similarities, author names, and author sizes.
    Return `top_n` authors as a dataframe.
    
    '''
    
    # Get similarities.
    sims = get_sims(model.get_author_topics(name))

    # Arrange author names, similarities, and author sizes in a list of tuples.
    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            
    # Make dataframe and retrieve top authors.
    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [48]:
get_table('Jonathan L. Sessler')
get_table('Jonathan L. Sessler', smallest_author=3)

,Author,Score,Size
4273,Jonathan L. Sessler,1.000000,6
7656,Ryong Ryoo,0.985974,1
4837,Kimberly H. Hartstein,0.972644,1
9552,Xue-Lu Ma,0.971740,1
3443,Hui-Min Gao,0.966999,1
6285,Mohamed I. Hashim,0.962918,1
9709,Ye-Fei Li,0.958315,1
3724,James M. Mayer,0.957289,2
5437,M. R. Ajayakumar,0.951670,1
384,Amanda J. Morris,0.943117,1


In [50]:
author_dict = {}
for a in author2doc:
    topic = [(topics[t[0]],t[1]) for t in model[a]]
    topic = [t for t in topic if (t[1] > 0.95)]
    #top3 = topic[0:3]
    author_dict[a] = [i[0] for i in topic]


In [52]:
#print(author_dict)
topic_author_list = {}
for t in topics:
    topic_author_list[t] = ''
    
print(topic_author_list)
for author, topic_list in author_dict.items():
    if topic_list is not None:
        for t in topic_list:
            topic_author_list[t]+=author+', '

#print(topic_author_list)

{'complex protein reaction': '', 'structure specie surface': '', 'drug tumor target': '', 'student medical research': '', 'associate factor infection': '', 'propose system model': '', 'visual task experiment': '', 'meta treatment trial': '', 'water material temperature': '', 'significant difference group': '', 'finding disease case': '', 'testing child test': '', 'participant care health': '', 'expression cancer gene': '', 'expression plant cell': ''}
{'complex protein reaction': 'Yunxiang Wu, Wilson C. Ho, Daniel A. Ciulla, Emily E. Robinson, Christoph Heinz, Benjamin M. Wolf, Padon Chuentragool, Shelley D. Minteer, Abaid Ullah Malik, Zhongda Pan, Hao Wang, Jinghua An, Matthew J. Harper, Phil S. Baran, Bryan D. Ellis, Shermin S. Goh, Biao Cheng, Maho Morita, Justin A. Modica, Shu-Shan Gao, Masanobu Uchiyama, Cathleen M. Crudden, Tomislav Rovis, Sarah E. Reisman, Jan-E. Bäckvall, Steven J. Malcolmson, Rei Kinjo, Yong Huang, Robert R. Knowles, Robert M. Waymouth, John F. Hartwig, Matthe

In [59]:
#[(topics[topic[0]],topic[1]) for topic in model[name]]
d = []


for topic, authors in topic_author_list.items():
    d.append({"topic": topic,
           "authors":authors})

jsonfile = open('output.json', 'w')
json.dump(d, jsonfile)

In [118]:
pd.DataFrame.from_dict(topic_author_list, orient='index')

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
complex protein reaction,Yunxiang Wu,Wilson C. Ho,Daniel A. Ciulla,Emily E. Robinson,Christoph Heinz,Benjamin M. Wolf,Padon Chuentragool,Shelley D. Minteer,Abaid Ullah Malik,Zhongda Pan,...,Clément Mazet,Dennis G. Hall,Guangbin Dong,Milan Mrksich,Neil K. Garg,James J. Cregg,Zhi-Cai Shi,Qiang Liu,David Milstein,Qi-Lin Zhou
structure specie surface,Min Wu,Xinyu Chen,Rongrong Yin,Tomoaki Sasaki,Xue-Sen Hou,Tsunetomo Yamada,Wenyao Zhang,Martina Delbianco,Hidetaka Kasai,Oscar E. Vilches,...,None,None,None,None,None,None,None,None,None,None
drug tumor target,Leyu Wang,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
student medical research,Young-Mee Lee,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
associate factor infection,Wale Kumlachew,Farrah C Liu,Berhanu Elfu Feleke,Eiichiro Iwata,Banchalem Nega Angore,Didi Wen,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
propose system model,Jiantao Liu,Wongvisarut Khuangsatung,Ben Bellekens,Fakhar Zaman,George Papazafeiropoulos,Jean Baratgin,James A. Dawson,Katarzyna Pędziwiatr,Pranati Maity,Thiago Magalhães,...,None,None,None,None,None,None,None,None,None,None
visual task experiment,Samuel A Mehr,Michele Fornaciai,Nina M Hanning,Dan Parker,Jan Skerswetat,Tomomi Koizumi,Motonori Yamaguchi,Ashley Lawrence,Pan Zhang,Hiroshi Ueda,...,None,None,None,None,None,None,None,None,None,None
meta treatment trial,Zhenwei Yu,Wen-Liang Yu,Huazhang Xiong,Jianjiang Ruan,Zhang Hao,Anita J Brogan,Ping Zhu,Chong Gao,Xiu-Hua Mao,Wenzhang Ge,...,None,None,None,None,None,None,None,None,None,None
water material temperature,Marina Krasovska,Tai Thanh Tran,Maria C Llinàs,Siying Tan,Te-Hui Wu,Yongjie Ge,Masaya Matsuki,Shuxing Bai,Jun Pan,Max M. Hansmann,...,None,None,None,None,None,None,None,None,None,None
significant difference group,Vanja P Nickovic,Yanjun Zhang,Emi Yuda,Hyun Gyu Cha,Hyun-Ho Choi,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
